In [2]:
using MIPVerify
using MAT
using PyCall
using ProgressMeter
py"""fc0_weight = [[ 0.1707, -0.0302,  0.0161,  0.1356, -0.1352,  0.1885,  0.0203, -0.0184,
         -0.0076,  0.0186,  0.1101, -0.1153,  0.1739, -0.2012,  0.0168,  0.0299,
         -0.0305, -0.0379, -0.0477,  0.1191, -0.0157, -0.0233,  0.0201, -0.0316,
         -0.0048,  0.0726,  0.1877,  0.1767, -0.1428,  0.1927,  0.1212],
        [ 0.1247,  0.0641, -0.2784, -0.2317, -0.1759,  0.1099, -0.0093,  0.1972,
          0.0068,  0.1691, -0.1538,  0.2999, -0.3243,  0.0016,  0.0719, -0.1651,
          0.0887, -0.0736,  0.1284, -0.2500, -0.1747, -0.0085, -0.0043,  0.1399,
          0.0938, -0.0844, -0.2148,  0.0534, -0.0094, -0.2103,  0.0619],
        [ 0.0513,  0.1101, -0.2102, -0.0749, -0.0802, -0.0455, -0.1288, -0.0427,
          0.2257, -0.0862,  0.0386,  0.1728, -0.2819,  0.1703,  0.0418,  0.0475,
         -0.1662,  0.0898,  0.0009, -0.2110, -0.0634, -0.1638,  0.1760, -0.1957,
         -0.2629,  0.0767, -0.1073,  0.0420, -0.0087, -0.2337, -0.0340]]"""
fc0_weight = py"fc0_weight"
fc0_weight= transpose(fc0_weight)
fc0_weight = convert(Array, fc0_weight)
fc0_manual = BinaryLinear(fc0_weight,ones(3))

┌ Info: Precompiling MIPVerify [e5e5f8be-2a6a-5994-adbb-5afbd0e30425]
└ @ Base loading.jl:1423
ERROR: LoadError: SystemError: opening file "/Users/rahul/Documents/Github/MIPVerify.jl/src/net_components/layers/binarize-checkpoint.jl": No such file or directory
Stacktrace:
  [1] systemerror(p::String, errno::Int32; extrainfo::Nothing)
    @ Base ./error.jl:174
  [2] #systemerror#68
    @ ./error.jl:173 [inlined]
  [3] systemerror
    @ ./error.jl:173 [inlined]
  [4] open(fname::String; lock::Bool, read::Nothing, write::Nothing, create::Nothing, truncate::Nothing, append::Nothing)
    @ Base ./iostream.jl:293
  [5] open
    @ ./iostream.jl:282 [inlined]
  [6] open(f::Base.var"#364#365"{String}, args::String; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ Base ./io.jl:328
  [7] open
    @ ./io.jl:328 [inlined]
  [8] read
    @ ./io.jl:436 [inlined]
  [9] _include(mapexpr::Function, mod::Module, _path::String)
    @ Base ./loading.jl:1249
 [10] include(mod::Mod

LoadError: Failed to precompile MIPVerify [e5e5f8be-2a6a-5994-adbb-5afbd0e30425] to /Users/rahul/.julia/compiled/v1.7/MIPVerify/jl_CW03Hv.

In [5]:
py"""fc1_weight = [[-0.0359, -0.2975, -0.2090],
        [ 0.0145,  0.1269, -0.1844],
        [ 0.0047,  0.4206, -0.1208]]"""
fc1_weight = py"fc1_weight"
fc1_weight = transpose(fc1_weight)
fc1_weight = convert(Array, fc1_weight)
fc1_manual = BinaryLinear(fc1_weight,ones(3))

BinaryLinear(3 -> 3)

In [6]:
py"""fc2_weight = [[ 0.0290,  0.0645, -0.2633]]
"""
fc2_weight = py"fc2_weight"
fc2_weight = transpose(fc2_weight)
fc2_weight = convert(Array, fc2_weight)
fc2_manual = BinaryLinear(fc2_weight,ones(1))

BinaryLinear(3 -> 1)

In [30]:
n1 = Sequential([
        fc0_manual,
        Binarize(),
        fc1_manual,
        Binarize(),
        fc2_manual,
        Binarize(),
    ],"example")

sequential net example
  (1) BinaryLinear(31 -> 3)
  (2) Binarize()
  (3) BinaryLinear(3 -> 3)
  (4) Binarize()
  (5) BinaryLinear(3 -> 1)
  (6) Binarize()


In [31]:
using PyCall
py"""import numpy as np
test_image = np.loadtxt("test_images.txt")
test_label = np.loadtxt("test_label.txt")
"""
test_image = py"test_image";
test_label = py"test_label";

In [34]:
function toBoolean(x)
    if x[1] == -1
        return 0
    else 
        return 1
    end
end
function frac_correct(nn, image, label)
    num_correct = 0.0;
    num_samples = 277;
    for sample_index in 1:num_samples
        input = test_image[sample_index,:];
        actual_label = test_label[sample_index,:][1];
        predicted_label = input |> n1 ;
        if actual_label == predicted_label
            num_correct += 1
        end
        # println(predicted_label)
    end
    return num_correct / num_samples
end

frac_correct (generic function with 1 method)

In [35]:
frac_correct(n1, test_image, test_label)

0.0

In [33]:
frac_correct(n1, test_image, test_label)

[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0]
[1.0

0.0

In [18]:
fc0_weight

31×3 Matrix{Float64}:
 1.0  1.0  1.0
 0.0  1.0  1.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 0.0  0.0  0.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  1.0  1.0
 1.0  1.0  0.0
 1.0  0.0  1.0
 0.0  1.0  1.0
 1.0  0.0  0.0
 ⋮         
 1.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  0.0  1.0
 0.0  1.0  0.0
 0.0  1.0  0.0
 1.0  0.0  1.0
 1.0  0.0  0.0
 1.0  1.0  1.0
 0.0  0.0  0.0
 1.0  0.0  0.0
 1.0  1.0  0.0

In [23]:
test_image[1,:]

31-element Vector{Float64}:
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
  ⋮
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0

In [ ]:
test

277-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0